# 脱落変数バイアス（OVB）

ここでは．共変量を追加することで，推定される効果が変化する仕組みについて確認する．  
この仕組みの解釈は「**セレクションバイアスの影響をより小さくするためにどのような共変量をモデルに追加するべきか？**」という疑問に対して，  
「**目的変数Yと介入変数Zに対して相関のある変数を加えるべき**」という解決策を与えてくれる．

ますは２つの回帰分析モデルAとBを考える．  
$$
    （モデルA）\; Y_i = \alpha_0 \:+ \alpha_1Z_i \:+ u_i \quad(1)　\\
    （モデルB）\; Y_i = \beta_0 \:+ \beta_1Z_i \:+ \beta_2X_{omit,i} \:+ e_i \quad(2)
$$

ここでは，モデルAが誤ったモデルで，モデルBが正しいモデルである．

この２つのモデルの差は共編量 $X_{omit,i}$ が追加されているか否かにあり，  
モデルBはセレクションバイアスの影響が取り除かれた結果を得られるとする．  
介入変数の効果は介入変数 $Z_i$ にかかるパラメータであるため，それぞれのモデルにて $\alpha_1 \; \beta_1$ で表される．  
モデルAはモデルBと比較すると $X_{omit,i}$が省略されている．  
なので，モデルAにおける誤差項$u_i$の中には，$\beta_2X_{omit,i}$ とモデルBにおける誤差項$e_i$が含まれていることになる．

$$
    u_i = \beta_2X_{omit,i}+e_i
$$

$\beta_2$ はモデルBにおいて推定される $X_{omit,i}$ と $Y_i$ の相関に当たるものである．  
本来は  $Z_i$ を $X_{omit,i}$ に対して回帰するが仮定するほうが正しいが， $Z_i$ と $X_{omit,i}$ の相関を議論しているので，  
$\gamma_1$ は $X_{omit,i}$ に対して $Z_i$ を回帰させたときに得られる回帰係数を得ても議論して問題ない．  
$X_{omit,i}$を含まない式を得たいので， 以下の式のように， $X_{omit,i}$ に対して $Z_i$ を回帰させる式を採択する．

$$
    X_{omit,i} = \gamma_o + \gamma_1Z_i + \epsilon_i \quad(3)
$$

モデルB　(2)式から $X_{omit,i}$ を消去するため， (2)へ(3)を代入する．

$$
    Y_i = \beta_0 + \beta_1Z_i + \beta_2(\gamma_0 + \gamma_1Z_i + \epsilon_i) \quad(4) \\
    \Leftrightarrow Y_i = \beta_0 + \beta_2\gamma_0 + (\beta_1 + \gamma_1\beta_2)Z_i + \epsilon_i \quad(4')
$$

モデルBはセレクションバイアスの影響がより小さい分析結果を得られるため，  
$X_{omit,i}$ は分析において必要な共変量だということになる．  
この $X_{omit,i}$ のような本来必要だがモデルから抜け落ちている変数を**脱落変数**と呼ぶ．  

このとき $X_{omit,i}$ を無視したモデルAにおいて介入効果を示すパラメータ$\alpha_1$は(4')を見ると，  

$$
    \alpha_1 = \beta_1 + \gamma_1\beta_2
$$

となることがわかる．

$\beta_1$ はモデルBにて推定される効果を示しており，  
セレクションバイアスがうまく取り除かれた結果である．  
このことから $\alpha_1$ は，モデルBにて推定される効果 $\beta_1$ に，  
何かしらの値をとる $\gamma_1\beta_2$を加えたものであることがわかる．

この $\gamma_1\beta_2$ は**脱落変数バイアス**（Omitted Variable Bias：OVB）と呼ばれる．  
モデルAを使って効果の分析を行った場合，得られる効果の推定結果は本来の効果にOVBを加えたものになってしまう．  
しかし， $X_{omit,i}$ を加えたモデルBの場合，効果の推定結果からはOVBが取り除かれ正しい効果が推定されることになる．  
モデルAとモデルBの比較は，必要な共変量がモデルに含まれない場合には推定される効果にはOVBが含まれていることを示すとともに，  
一方ではそのような変数をモデルに加えることでOVBの影響を取り除くことが可能であるという回帰分析の仕組みを説明している．

**図解**
![回帰分析におけるバイアス_1](images/回帰分析におけるバイアス_1.png)

つまり，この $\beta_1$ のほかに余分にあるバイアス $\gamma_1\beta_2$ は， $X_{omit}$と $Y$ の相関に，  
$Z$ と $X_{omit}$ の相関を掛けたものということになる．  
直感的には，省略された共変量 $X_{omit}$ が $Y$ に対して与えるような影響が，  
$X_{omit}$ と $Z$ の相関を通して $Z$ の効果として表われているように見えているということになる．

以上のことから，OVBを考えると，興味のあるパラメータ以外の有意差検定の結果を考慮して，  
共変量を選択するようなことはむしろ害悪である可能性がある．

# RによるOVBの確認

ここでは以下の２つの回帰式におけるOVBの値を確認する．
$$
    Spend_i = \alpha_0 + \alpha_1treatment_i + \alpha_2recency_i + \alpha_3channel_i + e_i （モデルA） \\
    Spend_i = \beta_0 + \beta_1treatment_i + \beta_2recency_i + \beta_3channel_i + \beta_4history_i + u_i （モデルB）
$$

AとBの差はhistoryという共変量がモデルに含まれているか否かにある．  
historyは過去の購入額を表す変数なので，売上に対して強い相関を持つ．  
今回利用するbiased_dataは作成する際にhistoryが300以上の場合には介入が起こりやすくなるようにデータを加工している．  
このことから，historyをモデルから外すことで，脱落変数バイアスが発生するようになっていることがわかる．

OVBの式に従うと値は以下のようになるはずである．
$$
    \alpha_1 - \beta_1 = \gamma_1\beta_4
$$

$\gamma_1$ は以下の回帰式によって推定された値である．

$$
    history_i = \gamma_0 + \gamma_1treatment_i + \gamma_2recency_i + \gamma_3channel_i + \epsilon_i （モデルC）
$$

この回帰式はモデルAで脱落している変数 $history_i$ に対して，  
モデルAに含まれている変数を使って回帰分析を行ったものである．

**Rコード**

```R
# broomの読み出し
libray(broom)

# モデル式のベクトルを用意
formula_vec <- c(
    spend ~ treatment + recency + channel, #モデルA
    spend ~ treatment + recency + channel + history, #モデルB
    history ~ treatment + channel + recency, #モデルC
)

# formulaに名前を付ける
names(formula_vec) <- paste("reg", LETTERS[1:3], sep = "_")

# モデル式のデータフレーム化
models <- formula_vec %>%
    enframe(name = "model_index", value = "formula")

# まとめて回帰分析を実行
df_models <- models %>%
    mutate(model = map(.x = formula, .f = lm, data = biased_data)) %>%
    mutate(lm_result = map(.x = model, .f = tidy))

# モデルの結果を整形
df_results <- df_models %>%
    mutate(formula = as.character(formula)) %>%
    select(formula, model_index, lm_result) %>%
    unnset(cols = c(lm_result))

# モデルA,B，Cでのtreatmentのパラメータ（回帰係数）を抜き出す
treatment_coef <- df_results %>%
    filter(term == "treatment") %>%
    pull(estimate)

# モデルBからhistoryのパラメータ（回帰係数）を抜き出す
history_coef <- df_results %>%
    filter(
        model_index == "reg_B",
        term == "history"
    ) %>%
    pull(estimate)

# OVBの確認
OVB <- history_coef * treatment_coef[3]
coef_gap <- treatment_coef[1] - treatment_coef[2]

OVB #beta_2 * gamma_1
coef_gap #alpha_1 - beta_1
```

**Pythonコード**

In [1]:
# ライブラリーimport
import numpy as np
import pandas as pd
from scipy.stats import uniform
import statsmodels.formula.api as smf

In [2]:
# データセット作成
df_email = pd.read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")

df_male = df_email.query("segment != 'Womens E-Mail'").copy()
df_male["treatment"] = np.where(df_male["segment"] == "Mens E-Mail", 1, 0)

obs_rate_c = 0.5
obs_rate_t = 0.5

df_biased_email = (df_male
                   .assign(
                       obs_rate_c=np.where(
                           (df_male["history"] > 300) | (df_male["recency"] < 6) | (df_male["channel"] == "Multichannel"),
                           obs_rate_c,
                           1
                       )
                   )
                   .assign(
                       obs_rate_t=np.where(
                           (df_male["history"] > 300) | (df_male["recency"] < 6) | (df_male["channel"] == "Multichannel"),
                           obs_rate_t,
                           1
                       )
                   )
                   .assign(random_number=uniform.rvs(size=len(df_male), random_state=46))
                   .query("(treatment == 0 & random_number < obs_rate_c) | (treatment == 1 & random_number < obs_rate_t)")
)

In [3]:
# モデルのformulaを用意
model_list = [
    "spend ~ treatment + recency + channel", #モデルA
    "spend ~ treatment + recency + channel + history", #モデルB
    "history ~ treatment + channel + recency", #モデルC
]
# 辞書内包表記でformulaに名前を付ける
model_dict = {f"reg_{idx}": formula for idx, formula in enumerate(model_list)}

# OLSでfitさせた結果をDataFrameに格納
df_model = pd.DataFrame(
    data=[smf.ols(formula=formula, data=df_biased_email).fit() for formula in model_dict.values()],
    index=[reg for reg in model_dict.keys()]
).T

In [4]:
# モデルA,B，Cでのtreatmentのパラメータ（回帰係数）を抜き出す
treatment_coef = [df_model[column][0].params["treatment"] for column in df_model.columns]

# モデルBからhistoryのパラメータ（回帰係数）を抜き出す
history_coef = df_model["reg_1"][0].params["history"]

In [5]:
# OVBの確認
OVB = history_coef * treatment_coef[2]
coef_gap = treatment_coef[0] - treatment_coef[1]

print(OVB)
print(coef_gap)

0.0015424165543142167
0.0015424165543151958


これにより，実際に共変量を追加したモデルとしなかったモデルにおいて，  
推定される効果の差がOVBの式の結果と一致することがわかった．  
つまり，共変量を追加することで推定される効果の値に変化が生じるのは，  
共変量を追加したことによってOVBが消失したことに由来していることがわかる．

# OVBが与えてくれる情報

$$
    \alpha_1 = \beta_1 + \gamma_1\beta_2
$$
$$
    X_{omit,i} = \gamma_1Z_i + \epsilon_i
$$

上式のようなOVBの式は，共変量が不十分なモデルの持つバイアス構造を表す．  
そしてその構造とは，バイアスの値は「脱落変数 $X_{omit}$ と $Z$ の関係」と，  
「脱落変数 $X_{omit}$ と目的変数 $Y$の関係」の掛け合わせになることを示していた．

これは，介入変数 $Z$ の決定に何らかの関連を持っていて，さらに目的変数 $Y$ との相関にあるような変数を，  
新たにモデルに含めるとバイアスを低減できるという重要な情報を与えてくれます．  
このような $Z$, $Y$ の両方に関係あるような変数のことを**交絡因子**と呼ぶ．

目的変数との相関が0ではない変数は，しばしば回帰モデルにおいて含まれるべき重要な存在とされることがある．  
これは $Y$ を予測することを目的にするのであれば正しい判断であるが，  
その変数と $Z$ との相関が0であるような場合にはOVBの値は0になるために，  
その変数をモデルに加えても，効果の推定値は変化しないということになる．  
また，このときにどちらかが0でなくとも非常に小さい値を保つ場合には，  
結果的には発生するバイアスは無視できるほどに小さくなる場合がある．  
よって， $Y$ と $Z$ どちらに対してもある程度相関があるような変数を加えることがよりバイアスを少ない効果推定を行う鍵である．

OVBの値が小さい，つまり $\beta_4$ および　$\gamma_1$ が小さいという状態は，  
その変数がもともと $Y$ および $Z$ と関係が薄い変数である場合もあるが，  
すでにモデルに含まれている変数の影響によって小さくなっているというケースもある．  
新たに追加しようとしてる変数と相関の強い変数をあらかじめ共変量としてモデルに加えている場合にはOVBの値は小さくなっており，  
その変数を加えても推定される効果は大きく変化しないことになる．  
これは $Z$ の割り振りに関係するであろうすべての変数を共変量としてモデルに加えなくともバイアスが十分に小さくなる可能性があることを示す．

またバイアスを発生させるような変数 $X_{omit}$ がデータとして手に入らない場合にも，  
その変数と $Y$ と $Z$ との関係がそれぞれ正になるか負になるかを考えることで，  
今得られている効果の推定結果が過小に評価されているか過大に評価されているかを想定できる．

# Conditional Independence Assumption

効果検証のための回帰分析における共変量の選択は，理想的にはモデルに含まれていない変数によるOVBがすべて0になる状態を目指す．  
これはモデルに含めた共変量で条件付けた時に，介入変数が $Y^{(1)}$ や $Y^{(0)}$ とは独立している状況になり，  
**CIA (Conditional Independence Assumption)** と呼ばれる．  
より直感的な解釈としては，共変量の値が同一のサンプルにおいて，  
介入 $Z$ はランダムに割り振られているのに等しい状態というものである．

**具体例**
![回帰分析におけるバイアス_2](images/回帰分析におけるバイアス_2.png)

CIAは以下のような式で表される．

$$
    \{Y_i^{(1)}, Y_i^{(0)}\} \perp Z_i|X_i
$$

回帰分析を行った結果として得た介入の効果を正しいと考える場合，  
そのモデルに投入した共変量は上記の仮定を満たすと想定することになる．  
つまり，回帰分析で推定した効果の値が本当に正しいのかを考える場合，  
CIAが満たされているか否かを考える必要がある．

# 変数の選び方とモデルの評価

## 共変量の選択とCIA

回帰分析において，分析者は以下のようなステップでモデルをつくることになる．  

----
> 
1. 介入の割り当てがどのようにして決定されるかを考える（e.g. 去年の購買が多い人にメールを送りそう，最近購入したばかりの人にメールを送りそう，etc...）
2. 想定される決定方法を表現できるような共変量を選択する（e.g. 昨年の購買量，直近の購買など）
3. 選択した共変量とYとの関係性を考慮してそれぞれの関数を決める

このとき，推定された効果の妥当性を主張するためには，  
作ったモデルの共変量がCIAを満たしていると考える必要がある．  
しかし，これを主張する上で2個問題がある．

## バイアスが評価できないという問題

得られた効果の推定値がどの程度バイアスを持っているかを評価する方法がないという問題がある．  
評価が可能な場合には，共変量の組み合わせを試しつつバイアスの値を評価し，  
それがより小さくなるような組み合わせを最終モデルとしたくなる．  
しかし，実際には評価ができないために，このようなプロセスは実行できない．  
OVBはモデル間のバイアスの変化を示すようなものであるため，  
残りのバイアスの大きさを示してくれるわけではない．  
以上のような事情から，基本的に分析者はセレクションバイアスがどのような理由によって発生し，  
それがどのような変数を含めることでコントロールが可能であり，  
どのような変数の組み合わせを用いることでCIAが満たされるのかを仮定する必要がある．

## 必要な共変量がデータにはないという問題

もう一つが，手持ちのデータに含まれる変数だけではなバイアスが十分に減らせない可能性があるという問題である．  
仮に推定結果のバイアスの値が評価できるとすると，データの中でも最もバイアスを減らし得る最適な共変量を選ぶことができる．  
しかしその場合においても，バイアスを完全になくすことはできない可能性がある．

e.g.
ある要因がバイアスを生むことを理解していても，それを表現するようなデータが定義できない場合や，定義できてもそのデータを入手できない場合があげられる．  

このとき，得られている効果の推定値がセレクションバイアスの影響を受けていることはわかっても，  
それらの変数をモデルに取り込んでバイアスを減らすことはできないことになる．  
手持ちのデータに必要な共変量が存在しない場合，基本的には回帰分析ではセレクションバイアスの影響を取り除いて効果を推定することは不可能ということである．  
この場合の回帰分析における推定結果は，得られた効果について投入した共編量については影響を取り除けているものの，データに存在しない変数によるOVBの影響を受けていることになる．